In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq as hq
import datetime as dt
import time
import warnings
import pmdarima as pm
import datetime as dt
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)

In [2]:
trace = pd.read_csv('Trace2021/AzureFunctionsInvocationTraceForTwoWeeksJan2021.txt')
trace["start_timestamp"] = trace["end_timestamp"] - trace["duration"]
trace = trace[['app','func','start_timestamp','end_timestamp','duration']]
trace = trace.rename(columns = {'app': 'app_id', 'func': 'func_id'}, inplace = False)
trace.sort_values(by=['start_timestamp'], ascending=False)

#append the date to the DF
newDate = []
for i in range(0,len(trace)):
    sd = dt.datetime(2021,1,31,0,0,0) # used to output incremental timestamp
    sd = sd + dt.timedelta(seconds = trace['start_timestamp'][i])
    #print(sd)
    newDate.append(sd)
    
#append IT to the DF
listIT = []
tempDict = {}
app_id = trace['app_id']
start_time = trace['start_timestamp']
for i in range(0,len(trace)):
    if tempDict.get(app_id[i]) == None:
        tempDict[app_id[i]] = start_time[i]
        listIT.append(0)
    else:
        IT = start_time[i] - tempDict.get(app_id[i])
        IT = np.floor(IT/60)
        tempDict[app_id[i]] = start_time[i]
        listIT.append(IT)
trace["date"] = newDate
trace["IT"] = listIT
trace.head()

,app_id,func_id,start_timestamp,end_timestamp,duration,date,IT
0,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,e3cdb48830f66eb8689cc0223514569a69812b77e6611e...,0.001491,0.079491,0.078,2021-01-31 00:00:00.001491,0.0
1,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,337cd24a7d5fd5c92460faee4ebe6a186a0eb322bd17b7...,0.003860,57.157860,57.154,2021-01-31 00:00:00.003860,0.0
2,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,48cc770d590d3c5a7691b3b4e9302f82ec3be5ddc2a037...,0.005478,59.130478,59.125,2021-01-31 00:00:00.005478,0.0
3,f274d71de386ccc77e4ca74766dbc485461c3053059d47...,3d2aee54a133509f16fb636d74128c2adcfcac71c6dcef...,0.016541,6.252541,6.236,2021-01-31 00:00:00.016541,0.0
4,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,68bbfd828223a505d7917339f4656c5f33ff93225cdb9d...,0.016824,0.066824,0.050,2021-01-31 00:00:00.016824,0.0


In [78]:
filter_rows = trace.iloc[1:40000]
sampleTest = filter_rows[['app_id','start_timestamp']]
first_row_start_time = sampleTest.iloc[0, 1]
sampleTest['start_timestamp'] = sampleTest['start_timestamp'] - first_row_start_time
sampleTest = sampleTest.reset_index(drop = True)
#append the date to the DF

    
#append IT to the DF
listIT = []
tempDict = {}
app_id = sampleTest['app_id']
start_time = sampleTest['start_timestamp']

for i in range(0,len(sampleTest)):
    if tempDict.get(app_id[i]) == None:
        tempDict[app_id[i]] = start_time[i]
        listIT.append(0)
    else:
        IT = start_time[i] - tempDict.get(app_id[i])
        IT = np.floor(IT/60)
        tempDict[app_id[i]] = start_time[i]
        listIT.append(IT)

sampleTest["IT"] = listIT
sampleTest.head()

incoming_invocations = sampleTest.values.tolist()
print(sampleTest.head(1))
print(sampleTest.tail(1))
sampleTest.head()

                                              app_id  start_timestamp   IT
0  1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...              0.0  0.0
                                                  app_id  start_timestamp   IT
39998  a594f92f84072b4cd031fe5283d1781a6e98f430696dec...     35560.052566  0.0


,app_id,start_timestamp,IT
0,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,0.000000,0.0
1,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,0.001618,0.0
2,f274d71de386ccc77e4ca74766dbc485461c3053059d47...,0.012681,0.0
3,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,0.012964,0.0
4,f7bfe5bc8d2a37a5c15986fbfc2c477a746e866adcb966...,0.018131,0.0


In [79]:
IT_more_than_240 = sampleTest.loc[sampleTest['IT'] > 240]
IT_more_than_240

,app_id,start_timestamp,IT
31386,a55aa23d7c2a7c801bc9e68afe4c90c86dc0486922e472...,27000.149741,330.0
34104,1661c8faccd5c7ce3ba6d326dc7fb1f14e74bc7d2c2f48...,28767.221582,300.0
34159,ca4fde7aa358b20b77fb6b6e65e1bd4d92093608481b81...,28800.051399,300.0
37049,938e7f49544b3293cd6cc7ec3e63e1751085cf5cb6a004...,31480.824567,464.0
38562,6e42a4e8ea1df408e059ba37d0e04255fd8385235b3102...,34418.361162,366.0
39595,fe016d40663e76e90a342e0a8348ff15248368dd21b1d8...,35100.060247,315.0
39857,c8c43e1a911f29e5506460a2fbef61ff39723d672f3b3b...,35369.080369,416.0


In [14]:
incoming_invocations = [['app1', 1], # cold start, new id
['app1', 301], # cold start, IT = 1min, keep alive = current time + 1min
['app1', 362], # cold start, IT = 1min, keep alive 1min = current time + 1min
['app1', 483],
['app1', 664]  
                       ] # warm start, 

In [5]:
df = pd.DataFrame(incoming_invocations)
df.rename(columns = {0:'app_id',1:'start_timestamp'}, inplace = True)
df

,app_id,start_timestamp
0,app1,1
1,app1,301
2,app1,362
3,app1,483
4,app1,664


In [66]:
incoming_invocations = [['app1', 1], # cold start, new id
['app1', 301], # cold start, IT = 1min, keep alive = current time + 1min
['app1', 362], # cold start, IT = 1min, keep alive 1min = current time + 1min
['app1', 483],
['app1', 664]  
                       ] # warm start, 

df = pd.DataFrame(incoming_invocations)
df.rename(columns = {0:'app_id',1:'start_timestamp'}, inplace = True)

#append IT to the DF
listIT = []
tempDict = {}
app_id = df['app_id']
start_time = df['start_timestamp']
for i in range(0,len(df)):
    if tempDict.get(app_id[i]) == None:
        tempDict[app_id[i]] = start_time[i]
        listIT.append(0)
    else:
        IT = start_time[i] - tempDict.get(app_id[i])
        IT = np.floor(IT/60)
        tempDict[app_id[i]] = start_time[i]
        listIT.append(IT)

df["IT"] = listIT
incoming_invocations = df.values.tolist()
df.head()


,app_id,start_timestamp,IT
0,app1,1,0.0
1,app1,301,5.0
2,app1,362,1.0
3,app1,483,2.0
4,app1,664,3.0


In [64]:
for i in range(0,100):
    applicationDict['app1'].append(2)

# Simulation functions

In [80]:
default_keep_alive_time = 10
OOB_threshold = 1
percentile_position_cut_off = 12

# when an incoming function has been invoked
def function_invoked(app_id,IT, mode, debug):
    # application id has not been seen before
    if applicationDict.get(app_id) == None:
        if debug:
            print("NEW app_id: {}".format(app_id))
            print("COLD START BECAUSE OF NEW APPLICATION")
        result.write("NEW app_id: {} \n".format(app_id))
        result.write("COLD START BECAUSE OF NEW APPLICATION \n")
        applicationDict[app_id] = []
        #applicationDict[app_id].append(dt.datetime.now()) # dont have to append cause IT already calculated
        countDict["default_keepalive"] += 1
        countDict["cold_from_new"] += 1
        return 0, default_keep_alive_time
    
    # application id is already in the dictionary
    else:

        if debug:
            print("OLD app_id: {} ".format(app_id))
            print('IDLE TIME (IT) for application is: {} mins'.format(IT))
        result.write("OLD app_id: {} \n".format(app_id))
        result.write('IDLE TIME (IT) for application is: {} \n'.format(IT))
        #add idle time into list
        applicationDict[app_id].append(IT)
        #applicationDict[app_id][0] = curTime
        
        if(mode == 'standard_keep_alive'):
            if debug:
                print('standard keep alive mode... returning default keep alive time')
            result.write('standard keep alive mode... returning default keep alive time \n')
            countDict["default_keepalive"] += 1
            return 0, default_keep_alive_time
        
        # if the idle time is more than 240mins, return predicted time as it is out of bounds (OOB)
        
        if (IT > 240):
            if debug:
                print('application is OOB with idle time(IT) > 240')
            result.write('application is OOB with idle time(IT) > 240 \n')
            
            if mode == 'hybrid_without_OOB_prediction':
                if debug:
                    print('application is OOB but mode is without OOB prediction, returning default keep alive time')
                result.write('application is OOB but mode is without OOB prediction, returning default keep alive time \n')
                countDict["default_keepalive"] += 1
                return 0, default_keep_alive_time
            
            return predictIntervalTiming(app_id,debug)   
        # add the IT into the dictionary and plot the histogram to determine the TTL duration (mins)
        else:
            return plotHistogram(app_id,debug)
        
#prediction of application IT if too many OOB
#if number of OOB is below threshold, return standard keepalive time to save cost = TTL 10mins
# pre warm time = predicted time * 0.85 (15% reduction of predicted time for buffer)
# keep alive time = predicted time * 0.30 (30% of predicted time)
def predictIntervalTiming(app_id,debug):
    #do prediction of OOB applications here
    appData = applicationDict[app_id]
    data_more_than_240 = [x for x in appData if x > 240]
    #print(appData,len(data_more_than_240),OOB_threshold,'lalala')
    if len(data_more_than_240) < OOB_threshold:
        print('Not enough OOB threshold of:',OOB_threshold,'returning default time =', default_keep_alive_time)
        result.write('Not enough OOB threshold of: {}, returning default time = {} \n'.format(OOB_threshold, default_keep_alive_time))
        countDict["default_keepalive"] += 1
        return 0, default_keep_alive_time
    
    model1 = pm.auto_arima(appData,stepwise= False, seasonal= False, suppress_warnings= True)
    option1_predict = model1.predict(n_periods=1)
    print('application OOB predicted time = ',option1_predict)
    result.write('application OOB predicted time = {} \n'.format(option1_predict))
    countDict["oob_prediction"] += 1
    return option1_predict[0]*0.85, option1_predict[0]*0.30

# hybrid histogram policy
# categorise the histogram into 240 different minutes bins, returning the bin with the highest amount of frequency
# in the event of same frequency, return the lowest number bin
def plotHistogram(app_id,debug):
    appData = applicationDict[app_id]
    
    # Plot histogram with 240 bins ranging from 0 to 240
    counts, bins, _ = plt.hist(appData, bins=240, range=(0, 240), density=False, edgecolor='black')
    
    #check if histogram is representative
    CV = calculate_CV(counts)
    
    if debug:
        print('appplication IT data is: {}'.format(appData))
        print('calculating CV = ',CV)
    #result.write('appplication IT data is: {} \n'.format(appData))
    result.write('CV = {} \n'.format(CV))
    
    if CV < 1:
        if debug:
            print('Histogram is not representative')
        result.write('Histogram is not representative \n')
        countDict["default_keepalive"] += 1
        return 0, default_keep_alive_time
    
    if debug:
        print('Histogram is representative')
    result.write('Histogram is representative \n')
    # Add labels and title
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram')

    # Show the plot
    #plt.show()
    plt.close()
    #show the array of IT
    #print(counts) 
    data = counts.tolist()

    countDict["histogram_keepalive"] += 1
    
    # the pre-warming list
    #head = data[:percentile_position_cut_off] 
    # the keep-alive list
    #tail = data[percentile_position_cut_off:]
    #pre_warm_time = head.index(max(head))
    #keep_alive_time = tail.index(max(tail))
    
    keep_alive_time = data.index(max(data))
    
    if keep_alive_time == 0:
        return 0, 1
    else:
        #TODO
        #split the keep alive time into [80,20], 80% pre warm -10% for buffer and 20% keep alive +10% for buffer
        head = (keep_alive_time * 0.8)
        tail = (keep_alive_time * 0.3)
        return head, tail

def check_for_cold_start(keep_alive_list, app_id,debug):
    index = next((i for i, (value1, value2) in enumerate(keep_alive_list) if value2 == app_id), None)
    
    if index is not None:
        if debug:
            print("WARM START")
        result.write("WARM START \n")
        countDict["warm"] += 1
        countDict["list_of_cold_or_warm_starts"].append(0)
    else:
        if debug:
            print("COLD START")
        result.write("COLD START \n")
        countDict["cold"] += 1
        countDict["list_of_cold_or_warm_starts"].append(1)
        
def update_keep_alive_element(keep_alive_list, new_time, app_id):
    # Find the index of the element you want to update
    index = next((i for i, (value1, value2) in enumerate(keep_alive_list) if value2 == app_id), None)

    if index is not None:
        # Update the value of the element
        keep_alive_list[index] = (new_time, keep_alive_list[index][1])
        # Rebuild the heap
        hq.heapify(keep_alive_list)
        keep_alive_list.sort()
        
    else:
        hq.heappush(keep_alive_list, (new_time, app_id))
        
def update_pre_warming_element(pre_warm_list, new_time, app_id, keep_alive_time):
    #Find the index of the element you want to update
    index = next((i for i, (value1, value2, value3) in enumerate(pre_warm_list) if value2 == app_id), None)
    if index is not None:
        # Update the value of the element
        pre_warm_list[index] = (new_time, pre_warm_list[index][1], keep_alive_time)
        # Rebuild the heap
        hq.heapify(pre_warm_list)
        pre_warm_list.sort()
    else:
        hq.heappush(pre_warm_list, (new_time, app_id, keep_alive_time))

def removeKeepAliveIfAny(keep_alive_list, app_id):
    index = next((i for i, (value1, value2) in enumerate(keep_alive_list) if value2 == app_id), None)
    
    if index is not None:
        keep_alive_list[index] = keep_alive_list[-1]
        keep_alive_list.pop()
        hq.heapify(keep_alive_list)
        

# Welford’s online algorithm (CV) to determine wheather a histogram is representive or not
def calculate_CV(app_bins):
    N = len(app_bins)
    M = 0
    S = 0

    for k in range(1, N+1):
        x = app_bins[k-1]
        oldM = M
        M = M + (x - M) / k
        S = S + (x - M) * (x - oldM)
    
    return S / (N - 1)
    

def printPreWarmList(debug):
    if debug:
        print("pre warm list: {}".format(pre_warm_list))
    result.write("pre warm list: {} \n".format(pre_warm_list))
    
def printKeepAliveList(debug):
    if debug:
        print("keep alive list: {}".format(keep_alive_list))
    result.write("keep alive list: {} \n".format(keep_alive_list))
    
def printExpiredPreWarmList(debug):
    if debug:
        print("PRE-WARMING: {} has expired".format(pre_warm_list[0][1]))
    result.write("PRE-WARMING: {} has expired \n".format(pre_warm_list[0][1]))
    
def printExpiredKeepAliveList(debug):
    if debug:
        print("KEEP-ALIVE: {} container has expired".format(keep_alive_list[0][1]))
    result.write("KEEP-ALIVE: {} container has expired \n".format(keep_alive_list[0][1]))

def printStats():
    print("Printing simulation results...")
    print("Number of cold starts:",countDict["cold"])
    print("Number of cold starts from first time invocation:",countDict["cold_from_new"])
    print("Number of warm starts:",countDict["warm"])
    print("Number of default keep alive:",countDict["default_keepalive"]) #not representative
    print("Number of histogram prediction:",countDict["histogram_keepalive"]) #representative
    print("Number of OOB:",countDict["oob"]) #out of 4hours histogram range
    print("Number of OOB arima prediction:",countDict["oob_prediction"])
    print("List of Cold Starts: ",countDict["list_of_cold_or_warm_starts"])
    
    result.write("Printing simulation results...\n")
    result.write("Number of cold starts: {} \n".format(countDict["cold"]))
    result.write("Number of cold starts from first time invocation: {} \n".format(countDict["cold_from_new"]))
    result.write("Number of warm starts: {} \n".format(countDict["warm"],"\n"))
    result.write("Number of default keep alive: {} \n".format(countDict["default_keepalive"]))
    result.write("Number of histogram prediction: {} \n".format(countDict["histogram_keepalive"]))
    result.write("Number of OOB: {} \n".format(countDict["oob"]))
    result.write("Number of OOB arima prediction: {} \n".format(countDict["oob_prediction"]))
    result.write("List of Cold Starts: {} \n".format(countDict["list_of_cold_or_warm_starts"]))
    

In [81]:
# dictionary to store all the app_id
applicationDict = {}
# list to store all the current keep alive application id
keep_alive_list = []

pre_warm_list = []

countDict = {
    "warm":0,
    "cold":0,
    "oob":0,
    "oob_prediction":0,
    "default_keepalive":0,
    "histogram_keepalive":0,
    "cold_from_new":0,
    "list_of_cold_or_warm_starts":[]
             }

In [82]:
def simulateWithoutTimer(invocation_list,mode,dbug):
    debug = dbug
    
    for app_id, ST, IT  in invocation_list:
        if debug:
            print("\nTRIGGERED: application has been invoked at {}s".format(ST))
        result.write("\nTRIGGERED: application has been invoked at {}\n".format(ST))
        
        
        # as long as there is still applications in the pre-warming list, we will keep checking
        while len(pre_warm_list) != 0:
            if ST <= pre_warm_list[0][0]:
                break

            # since the TTL list is a heap queue, the first position is always the earliest terminated one
            # so we check if first element is pending expiry, then remove from heap queue
            check_expiry = pre_warm_list[0][0]
            # check if current simulation time has caught up with the application's TTL timing
            if ST >= check_expiry:

                printExpiredPreWarmList(debug)
                pre_warm_timer, app_id, keep_alive_timer = hq.heappop(pre_warm_list)

                #keep alive time
                if ST <= keep_alive_timer:
                    if debug:
                        print("Adding application into keep alive list...")
                    result.write("Adding application into keep alive list...")

                    update_keep_alive_element(keep_alive_list, keep_alive_timer, app_id)
                    printPreWarmList(debug)
                    printKeepAliveList(debug)

        # as long as there is still applications in the keep alive list, we will keep checking
        while len(keep_alive_list) != 0:
            if ST <  keep_alive_list[0][0]:
                break
                
            # since the TTL list is a heap queue, the first position is always the earliest terminated one
            # so we check if first element is pending expiry, then remove from heap queue
            check_expiry = keep_alive_list[0][0]
            # check if current simulation time has caught up with the application's TTL timing
            if ST >= check_expiry:
                printExpiredKeepAliveList(debug)
                hq.heappop(keep_alive_list)
                printKeepAliveList(debug)
                
                

        
        pre_warm_time, keep_alive_time = function_invoked(app_id,IT,mode,debug)
        if debug:
            print("PRE WARM:",pre_warm_time, " KEEP ALIVE:",keep_alive_time)
        result.write("PRE WARM: {}  KEEP ALIVE: {} \n".format(pre_warm_time,keep_alive_time))

        check_for_cold_start(keep_alive_list, app_id ,debug)
        
        if(pre_warm_time > 0):
            pre_warm_timer = ST + dt.timedelta(minutes = pre_warm_time).total_seconds()
            alive_timer = ST + dt.timedelta(minutes = pre_warm_time).total_seconds() + dt.timedelta(minutes = keep_alive_time).total_seconds()
            if debug:
                print("pre warm until ",pre_warm_timer)
                print("keep alive time is: {} mins".format(keep_alive_time))
            update_pre_warming_element(pre_warm_list, pre_warm_timer, app_id, alive_timer)
            printPreWarmList(debug)
            removeKeepAliveIfAny(keep_alive_list, app_id)
            printKeepAliveList(debug)
        # if pre warming time = 0
        else:
            if(keep_alive_time > 0):
                alive_timer = ST + dt.timedelta(minutes = keep_alive_time).total_seconds()
                if debug:
                    print("keep alive until: {}".format(alive_timer))
                update_keep_alive_element(keep_alive_list, alive_timer, app_id)
                printKeepAliveList(debug)

        
        
                
                
    print("\n\n==end of run==")
    printStats()
    result.write("==end of run==\n")
    return countDict

In [ ]:
result = open("test.txt","w") # output logs into a txt file as data is large
simulateWithoutTimer(incoming_invocations,"123",0)
result.close()

In [12]:
a = 600.00 + dt.timedelta(minutes = 1).total_seconds()
a

660.0

In [50]:
len(pre_warm_list)

0